In [8]:
# Import de bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [17]:
# Datasets de treino e teste
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

# Exibição dos datasets
print("Visão do dataset 'train.csv':")
print(train_df.head())
print("\nVisão do dataset 'test.csv':")
print(test_df.head())

Visão do dataset 'train.csv':
   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  Sal

In [10]:
# Estratégia para tratar valores faltantes nas colunas numéricas
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
train_df[numeric_cols] = train_df[numeric_cols].fillna(train_df[numeric_cols].mean())

common_numeric_cols = [col for col in numeric_cols if col in test_df.columns]
test_df[common_numeric_cols] = test_df[common_numeric_cols].fillna(test_df[common_numeric_cols].mean())

# Seleção de características e alvo
features = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
target = 'SalePrice'
X = train_df[features]
y = train_df[target]

In [11]:
# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Padronizar as características
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Criação e treino do modelo de Árvore de Decisão
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

# Previsões no conjunto de teste
predictions = model.predict(X_test)

In [12]:
# Avaliação do desempenho do conjunto de teste
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print('Mean Squared Error:', mse)
print('Mean Absolute Error:', mae)
print('R2 Score:', r2)

# Ajuste de parâmentros com validação cruzada
scores = cross_val_score(model, X_train, y_train, cv=5)
cross_val_r2 = np.mean(scores)
print('Cross-validated R2:', cross_val_r2)

# Previsão dos preços das casas
X_kaggle_test = test_df[features]
X_kaggle_test = scaler.transform(X_kaggle_test)
final_predictions = model.predict(X_kaggle_test)

Mean Squared Error: 1699691152.957382
Mean Absolute Error: 26708.171232876713
R2 Score: 0.7784069272875559
Cross-validated R2: 0.6411315543236578


In [16]:
# Previsões finais
print('\nSeleção de 15 casas:')
for i in range(15):  # Seleção das 15 primeiras casas
    house_features = test_df.iloc[i][features].to_dict()  # Características da casa
    print(f'ID: {test_df["Id"].iloc[i]}, Preço previsto: {final_predictions[i]}, Características: {house_features}')


Seleção de 15 casas:
ID: 1461, Preço previsto: 109500.0, Características: {'OverallQual': 5, 'GrLivArea': 896, 'GarageCars': 1.0, 'TotalBsmtSF': 882.0, 'FullBath': 1, 'YearBuilt': 1961}
ID: 1462, Preço previsto: 157000.0, Características: {'OverallQual': 6, 'GrLivArea': 1329, 'GarageCars': 1.0, 'TotalBsmtSF': 1329.0, 'FullBath': 1, 'YearBuilt': 1958}
ID: 1463, Preço previsto: 175000.0, Características: {'OverallQual': 5, 'GrLivArea': 1629, 'GarageCars': 2.0, 'TotalBsmtSF': 928.0, 'FullBath': 2, 'YearBuilt': 1997}
ID: 1464, Preço previsto: 192000.0, Características: {'OverallQual': 6, 'GrLivArea': 1604, 'GarageCars': 2.0, 'TotalBsmtSF': 926.0, 'FullBath': 2, 'YearBuilt': 1998}
ID: 1465, Preço previsto: 245500.0, Características: {'OverallQual': 8, 'GrLivArea': 1280, 'GarageCars': 2.0, 'TotalBsmtSF': 1280.0, 'FullBath': 2, 'YearBuilt': 1992}
ID: 1466, Preço previsto: 188000.0, Características: {'OverallQual': 6, 'GrLivArea': 1655, 'GarageCars': 2.0, 'TotalBsmtSF': 763.0, 'FullBath': 2, 